# Deploy NVIDIA NIM on Amazon SageMaker

NVIDIA NIM, a component of NVIDIA AI Enterprise, enhances your applications with the power of state-of-the-art large language models (LLMs), providing unmatched natural language processing and understanding capabilities. Whether you're developing chatbots, content analyzers, or any application that needs to understand and generate human language, NVIDIA NIM for LLMs has you covered.

In this example we show how to deploy `NVIDIA Nemotron Nano 9b v2` with NIM on Amazon SageMaker.

<div class="alert alert-block alert-info">
<b>IMPORTANT:</b> To run NIM on SageMaker you will need to have your NGC API KEY because it's required to access NGC resources. Check out <a href="https://build.nvidia.com/meta/llama3-70b?signin=true"> this LINK</a> to learn how to get NGC API KEY. 
</div>

Please check out the [NIM LLM docs](https://docs.nvidia.com/nim/large-language-models/latest/introduction.html) for more information.

**⚠️ Disclaimer**

Reasoning models often require longer inference times, which may exceed the default 60-second timeout limit for **AWS SageMaker's non-streaming endpoints**. This notebook shows examples for both the streaming and non-streaming endpoints 

To avoid inference failures due to timeout:
- It is **recommended** to use a **SageMaker streaming endpoint** for this model.
- If your use case **requires** using a **non-streaming endpoint**, you must first contact **AWS Support** to request an increased timeout limit for your **AWS Account and Region** to avoid unexpected errors.


## Setup

Installs the dependencies and setup roles required to package the model and create SageMaker endpoint. 

In [ ]:
import boto3, json, sagemaker, time, os
from sagemaker import get_execution_role
from pathlib import Path

sess = boto3.Session()
sm = sess.client("sagemaker")
sagemaker_session = sagemaker.Session(boto_session=sess)
role = get_execution_role()
client = boto3.client("sagemaker-runtime")
region = sess.region_name
sts_client = sess.client('sts')
account_id = sts_client.get_caller_identity()['Account']

### Define Arguments

In [ ]:
public_nim_image = "public.ecr.aws/nvidia/nim:nvidia-nemotron-nano-9b-v2-1.12"
nim_model = "nvidia-nemotron-nano-9b-v2"
sm_model_name = "nvidia-nemotron-nano-9b-v2"
instance_type = "ml.g5.2xlarge"
payload_model = "nvidia/nvidia-nemotron-nano-9b-v2"

### NIM Container

We first pull the NIM image from public ECR and then push it to private ECR repo within your account for deploying on SageMaker endpoint. Note:
  - NIM ECR image is currently available only in `us-east-1` region
  - You must have `ecr:CreateRepository` and appropriate push permissions associated with your execution role

In [ ]:
import subprocess

# Get AWS account ID
result = subprocess.run(['aws', 'sts', 'get-caller-identity', '--query', 'Account', '--output', 'text'], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

if result.returncode != 0:
    print(f"Error getting AWS account ID: {result.stderr}")
else:
    account = result.stdout.strip()
    print(f"AWS account ID: {account}")

bash_script = f"""
echo "Public NIM Image: {public_nim_image}"
docker pull {public_nim_image}


echo "Resolved account: {account}"
echo "Resolved region: {region}"

nim_image="{account}.dkr.ecr.{region}.amazonaws.com/{nim_model}"

# Ensure the repository name adheres to AWS constraints
repository_name=$(echo "{nim_model}" | tr '[:upper:]' '[:lower:]' | tr -cd '[:alnum:]._/-')

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "$repository_name" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "$repository_name" > /dev/null
fi

# Get the login command from ECR and execute it directly
aws ecr get-login-password --region {region} | docker login --username AWS --password-stdin "{account}.dkr.ecr.{region}.amazonaws.com"

docker tag {public_nim_image} $nim_image
docker push $nim_image
echo -n $nim_image
"""
nim_image=f"{account}.dkr.ecr.{region}.amazonaws.com/{nim_model}"
# Run the bash script and capture real-time output
process = subprocess.Popen(bash_script, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

while True:
    output = process.stdout.readline()
    if output == b'' and process.poll() is not None:
        break
    if output:
        print(output.decode().strip())

stderr = process.stderr.read().decode()
if stderr:
    print("Errors:", stderr)


We print the private ECR NIM image in your account that we will be using for SageMaker deployment. 
- Should be similar to  `"<ACCOUNT ID>.dkr.ecr.<REGION>.amazonaws.com/<NIM_MODEL>:latest"`

In [ ]:
print(nim_image)

### Create SageMaker Endpoint

**Before proceeding further, please set your NGC API Key.**

In [ ]:
# SET ME
NGC_API_KEY = None

In [ ]:
assert NGC_API_KEY is not None, "NGC API KEY is not set. Please set the NGC_API_KEY variable. It's required for running NIM."

We define sagemaker model from the NIM container making sure to pass in **NGC_API_KEY**

In [ ]:
container = {
    "Image": nim_image,
    "Environment": {
        "NGC_API_KEY": NGC_API_KEY,
        "NIM_MAX_NUM_SEQS": "8", # Set for smaller GPUs such as A10G (g5.xxxx instances) can remove for larger GPUs
        "NIM_MAMBA_SSM_CACHE_DTYPE": "auto" # Set for smaller GPUs such as A10G (g5.xxxx instances) can remove for larger GPUs
    }
}
create_model_response = sm.create_model(
    ModelName=sm_model_name, ExecutionRoleArn=role, PrimaryContainer=container
)

print("Model Arn: " + create_model_response["ModelArn"])

Next we create endpoint configuration, here we are deploying the LLama3-70B model on the specified instance type.

In [ ]:
endpoint_config_name = sm_model_name

create_endpoint_config_response = sm.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "InstanceType": instance_type,
            "InitialVariantWeight": 1,
            "InitialInstanceCount": 1,
            "ModelName": sm_model_name,
            "VariantName": "AllTraffic",
            "ContainerStartupHealthCheckTimeoutInSeconds": 1800,
            "InferenceAmiVersion": "al2-ami-sagemaker-inference-gpu-2"
        }
    ],
)

print("Endpoint Config Arn: " + create_endpoint_config_response["EndpointConfigArn"])

Using the above endpoint configuration we create a new sagemaker endpoint and wait for the deployment to finish. The status will change to InService once the deployment is successful.

In [ ]:
endpoint_name = sm_model_name

create_endpoint_response = sm.create_endpoint(
    EndpointName=endpoint_name, EndpointConfigName=endpoint_config_name
)

print("Endpoint Arn: " + create_endpoint_response["EndpointArn"])

In [ ]:
resp = sm.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print("Status: " + status)

while status == "Creating":
    time.sleep(60)
    resp = sm.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]
    print("Status: " + status)

print("Arn: " + resp["EndpointArn"])
print("Status: " + status)

## Run Inference

Once we have the endpoint's status as `InService` we can use a sample text to do a chat completion inference request using json as the payload format. For inference request format, currently NIM on SageMaker supports the OpenAI API chat completions inference protocol. For explanation of supported parameters please see [this link](https://platform.openai.com/docs/api-reference/chat). 

<div class="alert alert-block alert-info">
<b>IMPORTANT:</b> Model name in inference request payload needs to be the name of NIM model. Please DON'T change it below. 
</div>

### Non Reasoning Mode

In [ ]:
payload = {
  "model": payload_model,
  "messages": [
    {   
      "role": "system",
      "content": "/no_think"
    },
    {
      "role":"user",
      "content":"Explain how a transformer neural network works."
    }
  ],
  "max_tokens": 3000

}


response = client.invoke_endpoint(
    EndpointName=endpoint_name, ContentType="application/json", Body=json.dumps(payload)
)

output = json.loads(response["Body"].read().decode("utf8"))
print(json.dumps(output, indent=2))

### Reasoning Mode

In [ ]:
payload = {
  "model": payload_model,
  "messages": [
    {   
      "role": "system",
      "content": "/think"
    },
    {
      "role":"user",
      "content": "Explain how a transformer neural network works"
    }
  ],
  "max_tokens": 3000
}


response = client.invoke_endpoint(
    EndpointName=endpoint_name, ContentType="application/json", Body=json.dumps(payload)
)

output = json.loads(response["Body"].read().decode("utf8"))
print(json.dumps(output, indent=2))

## Streaming inference

NIM on SageMaker also supports streaming inference and you can enable that by setting **`"stream"` as `True`** in the payload and by using [`invoke_endpoint_with_response_stream`](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker-runtime/client/invoke_endpoint_with_response_stream.html) method.

### Non Reasoning Mode

In [ ]:
payload = {
  "model": payload_model,
  "messages": [
    {   
      "role": "system",
      "content": "/no_think"
    },
    {
      "role":"user",
      "content":"Explain how a transformer neural network works."
    }
  ],
  "max_tokens": 3000,
  "stream": True

}

response = client.invoke_endpoint_with_response_stream(
    EndpointName=endpoint_name,
    Body=json.dumps(payload),
    ContentType="application/json",
    Accept="application/jsonlines",
)

We have some postprocessing code for the streaming output.

In [ ]:
event_stream = response['Body']
accumulated_data = ""
start_marker = 'data:'
end_marker = '"finish_reason":null}]}'

for event in event_stream:
    try:
        payload = event.get('PayloadPart', {}).get('Bytes', b'')
        if payload:
            data_str = payload.decode('utf-8')

            accumulated_data += data_str

            # Process accumulated data when a complete response is detected
            while start_marker in accumulated_data and end_marker in accumulated_data:
                start_idx = accumulated_data.find(start_marker)
                end_idx = accumulated_data.find(end_marker) + len(end_marker)
                full_response = accumulated_data[start_idx + len(start_marker):end_idx]
                accumulated_data = accumulated_data[end_idx:]

                try:
                    data = json.loads(full_response)
                    content = data.get('choices', [{}])[0].get('delta', {}).get('content', "")
                    if content:
                        print(content, end='', flush=True)
                except json.JSONDecodeError:
                    continue
    except Exception as e:
        print(f"\nError processing event: {e}", flush=True)
        continue

### Reasoning Mode

In [ ]:
payload = {
  "model": payload_model,
  "messages": [
    {   
      "role": "system",
      "content": "/think"
    },
    {
      "role":"user",
      "content":"Explain how a transformer neural network works."
    }
  ],
  "max_tokens": 3000,
  "stream": True

}

response = client.invoke_endpoint_with_response_stream(
    EndpointName=endpoint_name,
    Body=json.dumps(payload),
    ContentType="application/json",
    Accept="application/jsonlines",
)

We have some postprocessing code for the streaming output.

In [ ]:
event_stream = response['Body']
accumulated_data = ""
start_marker = 'data:'
end_marker = '"finish_reason":null}]}'

for event in event_stream:
    try:
        payload = event.get('PayloadPart', {}).get('Bytes', b'')
        if payload:
            data_str = payload.decode('utf-8')

            accumulated_data += data_str

            # Process accumulated data when a complete response is detected
            while start_marker in accumulated_data and end_marker in accumulated_data:
                start_idx = accumulated_data.find(start_marker)
                end_idx = accumulated_data.find(end_marker) + len(end_marker)
                full_response = accumulated_data[start_idx + len(start_marker):end_idx]
                accumulated_data = accumulated_data[end_idx:]

                try:
                    data = json.loads(full_response)
                    content = data.get('choices', [{}])[0].get('delta', {}).get('content', "")
                    if content:
                        print(content, end='', flush=True)
                except json.JSONDecodeError:
                    continue
    except Exception as e:
        print(f"\nError processing event: {e}", flush=True)
        continue

## Agent implementation with Tool Calling

### Introduction

Based on the user input, the agent invokes **a tool from the pool of available tools**. The agent will decide and invoke the required **tool(s)** to get the response back to the user.

In [ ]:
class ToolError(Exception):
    pass

### Define tools for the agents

In [ ]:
def get_current_weather(input) -> int:
    """Get the current temperature from a city, in Fahrenheit"""
    
    city = input["city"].lower()
    country = input["country"].lower()
    
    # Hardcoded temperature data
    weather = {
        "us": {
            "new york": 82,
            "los angeles": 76,
            "chicago": 70
        },
        "gb": {
            "london": 68
        },
        "ca": {
            "toronto": 65,
            "vancouver": 60
        }
    }

    # Look up the temperature
    try:
        return weather[country][city]
    except KeyError:
        raise ValueError(f"No temperature found for {city.title()}, {country.upper()}")

def get_difference(input) -> int:
    """Get the difference between two numbers"""
    
    minuend = input["minuend"]
    subtrahend = input["subtrahend"]
    return minuend - subtrahend


tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current temperature from a city, in Fahrenheit",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "City"
                    },
                    "country": {
                        "type": "string",
                        "description": "Country Code (e.g. US, GB, CA)"
                    }
                },
                "required": ["city", "country"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_difference",
            "description": "Get the difference between two numbers",
            "parameters": {
                "type": "object",
                "properties": {
                    "minuend": {
                        "type": "integer",
                        "description": "The number from which another number is to be subtracted"
                    },
                    "subtrahend": {
                        "type": "integer",
                        "description": "The number to be subtracted"
                    }
                },
                "required": ["minuend", "subtrahend"]
            }
        }
    }
]

### Define helper function to call SageMaker

In [ ]:
def call_sagemaker(messages, tools):
    """Call SageMaker endpoint with OpenAI API format"""
    
    payload = {
        "model": payload_model,
        "messages": messages,
        "max_tokens": 2000,
        "temperature": 0,
        "tool_choice": "auto",
        "tools": tools
    }
    
    response = client.invoke_endpoint(
        EndpointName=endpoint_name,
        ContentType="application/json",
        Body=json.dumps(payload)
    )
    
    # Parse response
    response_body = json.loads(response['Body'].read().decode('utf-8'))
    return response_body

### Define helper function to invoke a given tool

In [ ]:
def get_tool_result(tool_call):
    """Execute a tool call and return the result"""
    tool_name = tool_call['function']['name']
    tool_args = json.loads(tool_call['function']['arguments'])
    
    print(f"Using tool `{tool_name}` with args `{tool_args}`")
    func = globals()[tool_name]
    try:
        return func(tool_args)  # Pass the full args dict, not just 'query'
    except Exception as e:
        raise ToolError(f"Something went wrong: {e}")

### Define function to handle the raw responses from SageMaker

In [ ]:
def handle_model_response(response):
    """Handle tool calls in the model response"""
    
    message = response['choices'][0]['message']
    
    # Check if there are tool calls
    if not message.get('tool_calls'):
        return None, message
    
    tool_messages = []
    
    for tool_call in message['tool_calls']:
        try:
            tool_result = get_tool_result(tool_call)
            
            tool_message = {
                "role": "tool",
                "tool_call_id": tool_call['id'],
                "content": json.dumps(tool_result)
            }
            tool_messages.append(tool_message)
            
        except ToolError as e:
            tool_message = {
                "role": "tool",
                "tool_call_id": tool_call['id'],
                "content": f"Error: {str(e)}"
            }
            tool_messages.append(tool_message)
    
    return tool_messages, message

### Define function that implements the Agent Loop till final response is received

In [ ]:
def run_agent(messages, tools):
    """Run the agent loop until completion"""
    MAX_LOOPS = 10
    loop_count = 0

    while loop_count < MAX_LOOPS:
        loop_count += 1
        
        # Call the model
        response = call_sagemaker(messages, tools)
        
        # Handle the response
        tool_messages, assistant_message = handle_model_response(response)
        
        # Add assistant message to conversation
        messages.append(assistant_message)
        
        # If no tool calls, we're done
        if tool_messages is None:
            final_output = assistant_message.get('content', '')
            break
        
        # Add tool results to conversation
        messages.extend(tool_messages)
    
    else:
        final_output = "Maximum loops reached"
    
    return messages, final_output

### Define agent executor

In [ ]:
def weather_agent_executor(input_prompt):
    
    system_prompt = system_prompt = """You are a helpful weather assistant. You have access to tools that return:
    - get_current_weather: Returns temperature in Fahrenheit for a given city
    - get_difference: Returns the numerical difference between two numbers
    
    When you make tool calls and receive results, use those results directly in your answer. 
    The tool results correspond exactly to the tool calls you made in the same order.

    Make the answer brief, do not mention tool calls in your answer
    """
    detailed_thinking = "/no_think"
    messages = [
        {
            "role": "system",
            "content": detailed_thinking
        },
        {
            "role": "system",
            "content": system_prompt
        },
        {
            "role": "user",
            "content": input_prompt
        }
    ]

    workflow, output = run_agent(messages, tools)
    return workflow, output

### Test agent

In [ ]:
workflow, output = weather_agent_executor("Where is it warmest: New york, london or Toronto? And by how much is it warmer than the other cities?")
print("\n========Output=============\n")
print(output)
print("\n========Tool calling details=============\n")
print(workflow)

## Terminate endpoint and clean up artifacts

In [ ]:
sm.delete_model(ModelName=sm_model_name)
sm.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
sm.delete_endpoint(EndpointName=endpoint_name)